## 卷積神經網路

In [1]:
! nvidia-smi

Mon May 11 06:42:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf

In [0]:
mnist = tf.keras.datasets.mnist

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
x_train.shape
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)


In [0]:
model = tf.keras.models.Sequential()

# 1. Convolution
model.add(tf.keras.layers.Conv2D(filters=16, 
                                      kernel_size=(3, 3), 
                                      padding = 'same', 
                                      input_shape = (28, 28, 1), 
                                      activation = 'relu'))

# 2. Max Pooling
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

# 3. Flattening
model.add(tf.keras.layers.Flatten())

# 4. Fully Connection
model.add(tf.keras.layers.Dense(units = 128, activation = 'relu')) 
model.add(tf.keras.layers.Dense(units = 10, activation = 'softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               401536    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 402,986
Trainable params: 402,986
Non-trainable params: 0
_________________________________________________________________


In [10]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer = 'adam', 
                   loss =loss_fn, 
                   metrics = ['accuracy'])

In [12]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [13]:
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/5
600/600 [==============================] - 2s 3ms/step - loss: 1.7030 - accuracy: 0.7643 - val_loss: 1.6006 - val_accuracy: 0.8646
Epoch 2/5
600/600 [==============================] - 2s 3ms/step - loss: 1.5250 - accuracy: 0.9412 - val_loss: 1.4912 - val_accuracy: 0.9741
Epoch 3/5
600/600 [==============================] - 2s 3ms/step - loss: 1.4881 - accuracy: 0.9762 - val_loss: 1.4859 - val_accuracy: 0.9779
Epoch 4/5
600/600 [==============================] - 2s 3ms/step - loss: 1.4812 - accuracy: 0.9825 - val_loss: 1.4819 - val_accuracy: 0.9805
Epoch 5/5
600/600 [==============================] - 2s 3ms/step - loss: 1.4770 - accuracy: 0.9862 - val_loss: 1.4802 - val_accuracy: 0.9825
